### 1. Motivation
#### What is your dataset?
We used three datasets related to Austin Animal Center:
- Austin_Animal_Center_Intakes_20250419.csv: records of animals entering the shelter, including intake dates and animal types.
- Austin_Animal_Center_Outcomes_20250419.csv: outcome records of animals leaving the shelter (e.g., adoption, euthanasia).
- shelter_geocoded_locations.csv: a manually geocoded dataset that maps official shelter addresses in Austin to their corresponding latitude and longitude coordinates, based on data from the Austin Animal Center's official shelter list.
- neighbour.json: downloaded from https://data.austintexas.gov/.

#### Why did you choose this/these particular dataset(s)?
The intake dataset offers a rich, time-stamped record of real-world animal shelter operations. With the help of manually geocoded shelter addresses, we were able to conduct meaningful spatial analysis. This multi-angle approach allows for an accessible and engaging visual story.

#### What was your goal for the end user's experience?
We want users to understand the patterns behind stray animal intake: when animals enter shelters most frequently, and which shelter regions are under more pressure. The visualizations should guide users from temporal overview to regional insights.

### 2. Basic stats
#### Data Cleaning & Preprocessing Steps
- Merged intake & outcome tables: According to Animal ID.Keeping duplicates for repeat visits.
- Date parsing: Converted intake_datetime to proper datetime format.
- Shelter matching: Linked each record to a shelter location based on the shelter_name or address field.
- Geolocation: Used shelter_geocoded_locations.csv to attach coordinates to each shelter site.
- Neighborhood mapping: Mapped coordinates to official Austin neighborhood polygons using spatial joins.
- Time aggregation: Grouped intake counts by month and by shelter region.
- Outliers: Removed records with missing animal type or unmatchable shelter data.

#### Dataset Statistics
- Intakes dataset rows: 126,234
- Outcomes dataset rows: 124,513
- Unique animal types: 5 (Dogs, Cats, Birds, Others, Small Mammals)
- Mapped shelter coordinates: Successfully matched to all listed shelters
- Time span: October 2013 to April 2025
- Number of mapped shelters: 12

### 3. Data Analysis

- We used Folium’s HeatMapWithTime to animate changes in geographic hotspots across Austin over time. Each frame represents a specific month from October 2013 to April 2025.

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime, Fullscreen, MeasureControl
import branca.colormap as cm

# === Step 1: Load data ===
df = pd.read_csv("merged_data.csv", parse_dates=["Intake Datetime"])
df = df.dropna(subset=["Latitude", "Longitude", "Intake Datetime"]).copy()
df = df[(df["Latitude"].between(-90, 90)) & (df["Longitude"].between(-180, 180))]
df["Month_Formatted"] = df["Intake Datetime"].dt.strftime("%Y-%m")

# === Step 2: Add weight ===
def add_weight(points_list):
    from collections import defaultdict
    point_counts = defaultdict(int)
    weighted = []
    for p in points_list:
        rounded = (round(p[0], 3), round(p[1], 3))
        point_counts[rounded] += 1
    for p in points_list:
        rounded = (round(p[0], 3), round(p[1], 3))
        weighted.append([p[0], p[1], min(point_counts[rounded] * 0.2, 1)])
    return weighted

time_index = sorted(df["Month_Formatted"].unique())
heat_data = [add_weight(df[df["Month_Formatted"] == m][["Latitude", "Longitude"]].values.tolist()) for m in time_index]

# === Step 3: Setup map ===
center = [df["Latitude"].median(), df["Longitude"].median()]
m = folium.Map(location=center, zoom_start=12, tiles="CartoDB Positron", control_scale=True)
MeasureControl(position="bottomright").add_to(m)

# === Step 4: Color gradient ===
colors = ["#c4e9f2", "#7fbbdd", "#f58b05", "#ffc22f"]
colormap = cm.LinearColormap(colors=colors, index=[0, 0.3, 0.6, 1], vmin=0, vmax=1, caption="Hotspot Density")
colormap.add_to(m)

# === Step 5: Heatmap with time ===
HeatMapWithTime(
    data=heat_data,
    index=time_index,
    radius=15,
    min_opacity=0.3,
    max_opacity=0.9,
    gradient={i / 3: colors[i] for i in range(4)},
    use_local_extrema=True,
    auto_play=True,
    display_index=True
).add_to(m)

# === Step 6: Title box ===
title_html = f"""
<div id="title-card" style="
    position: absolute;
    top: 20px;
    left: 20px;
    z-index: 9999;
    background-color: rgba(255,255,255,0.85);
    padding: 10px 15px;
    border-radius: 8px;
    font-family: sans-serif;
    box-shadow: 0 0 8px rgba(0,0,0,0.1);
    max-width: 300px;
    font-size: 13px;
">
    <h4 style="margin: 0; font-size: 1em;"><b>Dynamic Hotspot Map of Animal Intakes</b></h4>
    <p style="margin: 2px 0;">Monthly distribution of animal intake hotspots</p>
    <p style="margin: 0;">Data range: {df["Intake Datetime"].min().strftime('%Y-%m')} to {df["Intake Datetime"].max().strftime('%Y-%m')}</p>
</div>
"""
m.get_root().html.add_child(folium.Element(title_html))

# === Step 7: Final JS fix using MutationObserver ===
custom_js = """
<script>
document.addEventListener("DOMContentLoaded", function () {
    const map = document.querySelector('.folium-map');
    if (map) map.style.position = 'relative';

    const observer = new MutationObserver(() => {
        const ctrl = document.querySelector('.leaflet-control-timecontrol');
        const target = document.querySelector('.leaflet-bottom.leaflet-left');

        if (ctrl && target && ctrl.querySelectorAll("button").length > 0) {
            // Remove all buttons except play/pause (index 1)
            const buttons = ctrl.querySelectorAll("button");
            buttons.forEach((btn, i) => {
                if (i !== 1) btn.remove();
            });

            // Remove fps control row
            const rows = ctrl.querySelectorAll("tr");
            if (rows.length > 1) rows[1].remove();

            // Move and style control
            target.appendChild(ctrl);
            Object.assign(ctrl.style, {
                position: 'absolute',
                left: '0',
                bottom: '0',
                margin: '20px',
                zIndex: '10000',
                background: 'white',
                borderRadius: '8px',
                padding: '8px',
                boxShadow: '0 0 6px rgba(0,0,0,0.2)',
                display: 'inline-block',
                maxWidth: '600px',
                overflow: 'hidden',
                transform: 'translateX(0%)'
            });

            observer.disconnect();
        }
    });

    observer.observe(document.body, { childList: true, subtree: true });
});
</script>
"""
m.get_root().html.add_child(folium.Element(custom_js))

# === Step 8: Fullscreen button ===
Fullscreen(position="topright").add_to(m)

# === Step 9: Save output ===
m.save("heatmap_final_clean_controls.html")
print("✅ Saved: heatmap_final_clean_controls.html")

Using a GeoJSON file of Austin neighborhoods, we mapped total intake counts per region and applied a yellow-to-blue gradient using Folium and D3 for visual contrast.

### 4. Genre
#### Which genre of data story did you use?
We used a Martini Glass narrative structure: The stem is a focused, linear presentation (time series → heatmap animation),followed by an open exploration body (static choropleth map for spatial analysis and comparison).

#### Visual Narrative tools used
- Graphical highlighting (e.g., saturation of heatmap points over time)
- Progressive reveal (the animated heatmap adds time dimension interactively)
- Visual grouping (e.g., color mapping in the choropleth for comparing regions)

#### Narrative Structure tools used
- Author-driven sequence at the start (clear framing of the problem through time trend)
- Reader-driven interaction in choropleth map and animation slider
- Multi-messaging via annotation blocks and map legends

###  5. Visualizations
We created three key visualizations to support our story:
- Monthly Intake Time Series
→ Highlights long-term trends and seasonal fluctuations
- Animated Heatmap of Found Locations
→ Shows how animal sightings spread and shift over time

- Choropleth Map by Neighborhood
→ Allows spatial comparison of intake burden across city districts

- Shelter Coverage & Intakes Map
→ Shows where intake hotspots align—or don’t—with current 5 km shelter coverage.

- Intake → Outcome Flow Sankey
→ Summarises how each intake source splits into final outcomes

### 6. Discussion
#### What went well?
The time series plot clearly visualizes trend and seasonality.The animated heatmap engages viewers while providing intuitive geographic context.The choropleth map makes it easy to spot disparities across neighborhoods.

#### What is still missing? What could be improved? Why?
We lost about 55% of geocoded addresses, which reduced the spatial accuracy. An interactive dashboard combining filters (e.g., by animal type, intake condition) would offer deeper insights. Labeling and overlays on maps could enhance interpretability for first-time viewers.

### 7. Contributions